In this notebook, we derive the group product and inverse tables for the single-qubit Clifford group, presented in terms of $C(e, s, x, w) \mathrel{:=} E^e S^s X^x \omega^w$ where $\omega^8 = 1$ and where $E \mathrel{:=} H S^3 \omega^3$.

# Preamble

In [1]:
import numpy as np
import qutip as qt

C:\Users\cgran\Anaconda3\envs\qdk-libraries-notes\lib\site-packages\qutip\__init__.py:169: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


In [2]:
X = qt.sigmax()
H = qt.qip.operations.hadamard_transform()
S = qt.qip.operations.phasegate(np.pi / 2)
omega = np.exp(1j * np.pi / 4)
E = H * (S ** 3) * (omega ** 3)
I = qt.qeye(2)

In [3]:
clifford_group = {
    (e, s, x, w): (E ** e) * (S ** s) * (X ** x) * (omega ** w)
    for e in range(3)
    for s in range(4)
    for x in range(2)
    for w in range(8)
}

# Group inverses

In [4]:
def find_inverse(clifford):
    for ((e, s, x, w), candidate) in clifford_group.items():
        if (candidate * clifford - I).norm() <= 1e-10:
            return (e, s, x, w)

In [5]:
inverses = {
    (e, s, x): find_inverse(clifford_group[(e, s, x, 0)])
    for e in range(3)
    for s in range(4)
    for x in range(2)
}

We export this as a Q# table:

In [6]:
clifford_group[(2, 3, 1, 2)] * clifford_group[(2, 1, 0, 6)]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.+1.j]
 [0.-1.j 0.+0.j]]

In [7]:
for idx, ((e, s, x), (e_inv, s_inv, x_inv, w_inv)) in enumerate(inverses.items()):
    clause = "if  " if idx == 0 else "elif"
    print(f"{clause} e == {e} and s == {s} and x == {x} {{ return SingleQubitClifford({e_inv}, {s_inv}, {x_inv}, {w_inv}); }}")

if   e == 0 and s == 0 and x == 0 { return SingleQubitClifford(0, 0, 0, 0); }
elif e == 0 and s == 0 and x == 1 { return SingleQubitClifford(0, 0, 1, 0); }
elif e == 0 and s == 1 and x == 0 { return SingleQubitClifford(0, 3, 0, 0); }
elif e == 0 and s == 1 and x == 1 { return SingleQubitClifford(0, 1, 1, 6); }
elif e == 0 and s == 2 and x == 0 { return SingleQubitClifford(0, 2, 0, 0); }
elif e == 0 and s == 2 and x == 1 { return SingleQubitClifford(0, 2, 1, 4); }
elif e == 0 and s == 3 and x == 0 { return SingleQubitClifford(0, 1, 0, 0); }
elif e == 0 and s == 3 and x == 1 { return SingleQubitClifford(0, 3, 1, 2); }
elif e == 1 and s == 0 and x == 0 { return SingleQubitClifford(2, 0, 0, 0); }
elif e == 1 and s == 0 and x == 1 { return SingleQubitClifford(2, 2, 1, 6); }
elif e == 1 and s == 1 and x == 0 { return SingleQubitClifford(1, 1, 0, 2); }
elif e == 1 and s == 1 and x == 1 { return SingleQubitClifford(1, 3, 0, 2); }
elif e == 1 and s == 2 and x == 0 { return SingleQubitClifford(2

In [8]:
clifford_group[1, 1, 1, 0] * clifford_group[1, 3, 0, 2]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]

In [9]:
clifford_group[1, 3, 0, 2] * clifford_group[1, 1, 1, 0]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]

# Group Products

Consider $C(e, s, x, w) \cdot C(e', s', x', w') = E^e S^s X^x \omega^w \cdot E^{e'} S^{s'} X^{x'} \omega^{w'} = E^e S^s X^x \cdot E^{e'} S^{s'} X^{w'} \omega^{w + w'}$.
We need to commute $E^{e'}$ past $S^{s} X^{x}$ in order to combine the $E$ factors together.

Doing so will give us a new Clifford, so we start by making a table of Clifford operators of the form $S^{s} X^x E^{e'} = E^{e''} S^{s''} X^{x''} \omega^{w''}$.

In [10]:
def find_sxe(s, x, ep):
    prod = (S ** s) * (X ** x) * (E ** ep)
    for ((e, s, x, w), candidate) in clifford_group.items():
        if (candidate - prod).norm() <= 1e-10:
            return (e, s, x, w)

In [11]:
sxe_table = {
    (s, x, ep): find_sxe(s, x, ep)
    for s in range(4)
    for x in range(2)
    for ep in range(3)
}

In [12]:
sxe_table

{(0, 0, 0): (0, 0, 0, 0),
 (0, 0, 1): (1, 0, 0, 0),
 (0, 0, 2): (2, 0, 0, 0),
 (0, 1, 0): (0, 0, 1, 0),
 (0, 1, 1): (1, 2, 0, 0),
 (0, 1, 2): (2, 2, 1, 6),
 (1, 0, 0): (0, 1, 0, 0),
 (1, 0, 1): (2, 3, 0, 6),
 (1, 0, 2): (1, 1, 1, 2),
 (1, 1, 0): (0, 1, 1, 0),
 (1, 1, 1): (2, 1, 0, 6),
 (1, 1, 2): (1, 3, 0, 4),
 (2, 0, 0): (0, 2, 0, 0),
 (2, 0, 1): (1, 2, 1, 6),
 (2, 0, 2): (2, 0, 1, 0),
 (2, 1, 0): (0, 2, 1, 0),
 (2, 1, 1): (1, 0, 1, 2),
 (2, 1, 2): (2, 2, 0, 2),
 (3, 0, 0): (0, 3, 0, 0),
 (3, 0, 1): (2, 1, 1, 4),
 (3, 0, 2): (1, 1, 0, 2),
 (3, 1, 0): (0, 3, 1, 0),
 (3, 1, 1): (2, 3, 1, 0),
 (3, 1, 2): (1, 3, 1, 0)}

In [13]:
for idx, ((s, x, ep), (epp, spp, xpp, wpp)) in enumerate(sxe_table.items()):
    clause = "if  " if idx == 0 else "elif"
    print(f"{clause} s == {s} and x == {x} and ep == {ep} {{ return SingleQubitClifford({epp}, {spp}, {xpp}, {wpp}); }}")

if   s == 0 and x == 0 and ep == 0 { return SingleQubitClifford(0, 0, 0, 0); }
elif s == 0 and x == 0 and ep == 1 { return SingleQubitClifford(1, 0, 0, 0); }
elif s == 0 and x == 0 and ep == 2 { return SingleQubitClifford(2, 0, 0, 0); }
elif s == 0 and x == 1 and ep == 0 { return SingleQubitClifford(0, 0, 1, 0); }
elif s == 0 and x == 1 and ep == 1 { return SingleQubitClifford(1, 2, 0, 0); }
elif s == 0 and x == 1 and ep == 2 { return SingleQubitClifford(2, 2, 1, 6); }
elif s == 1 and x == 0 and ep == 0 { return SingleQubitClifford(0, 1, 0, 0); }
elif s == 1 and x == 0 and ep == 1 { return SingleQubitClifford(2, 3, 0, 6); }
elif s == 1 and x == 0 and ep == 2 { return SingleQubitClifford(1, 1, 1, 2); }
elif s == 1 and x == 1 and ep == 0 { return SingleQubitClifford(0, 1, 1, 0); }
elif s == 1 and x == 1 and ep == 1 { return SingleQubitClifford(2, 1, 0, 6); }
elif s == 1 and x == 1 and ep == 2 { return SingleQubitClifford(1, 3, 0, 4); }
elif s == 2 and x == 0 and ep == 0 { return SingleQu

Using this table, we now have an operator of the form $E^e S^s X^x \cdot E^{e'} S^{s'} X^{w'} = E^{e} E^{e''} S^{s''} X^{x''} S^{s'} X^{x'} \omega^{w''} = E^{e + e''} S^{s''} X^{x''} S^{s'} X^{x'} \omega^{w''}$.
Considering the factor $S^{s''} X^{x''} S^{s'} X^{x'}$, we can simplify by finding $s'''$, $x'''$, and $w'''$ such that $S^{s''} X^{x''} S^{s'} X^{x'} = S^{s''} S^{s'''} X^{x'''} X^{x'} \omega^{w'''} = S^{s'' + s'''} X^{x''' + x'} \omega^{w'''}$.

In [14]:
from itertools import product

In [15]:
def find_sx(xpp, sp):
    prod = (X ** xpp) * (S ** sp)
    for (sppp, xppp, wppp) in product(range(4), range(2), range(8)):
        if (clifford_group[0, sppp, xppp, wppp] - prod).norm() <= 1e-10:
            return (sppp, xppp, wppp)

In [16]:
sx_table = {
    (xpp, sp): find_sx(xpp, sp)
    for xpp in range(2)
    for sp in range(4)
}

In [17]:
sx_table

{(0, 0): (0, 0, 0),
 (0, 1): (1, 0, 0),
 (0, 2): (2, 0, 0),
 (0, 3): (3, 0, 0),
 (1, 0): (0, 1, 0),
 (1, 1): (3, 1, 2),
 (1, 2): (2, 1, 4),
 (1, 3): (1, 1, 6)}

In [18]:
for idx, ((xpp, sp), (sppp, xppp, wppp)) in enumerate(sx_table.items()):
    clause = "if  " if idx == 0 else "elif"
    print(f"{clause} xpp == {xpp} and sp == {sp} {{ return SingleQubitClifford(0, {sppp}, {xppp}, {wppp}); }}")

if   xpp == 0 and sp == 0 { return SingleQubitClifford(0, 0, 0, 0); }
elif xpp == 0 and sp == 1 { return SingleQubitClifford(0, 1, 0, 0); }
elif xpp == 0 and sp == 2 { return SingleQubitClifford(0, 2, 0, 0); }
elif xpp == 0 and sp == 3 { return SingleQubitClifford(0, 3, 0, 0); }
elif xpp == 1 and sp == 0 { return SingleQubitClifford(0, 0, 1, 0); }
elif xpp == 1 and sp == 1 { return SingleQubitClifford(0, 3, 1, 2); }
elif xpp == 1 and sp == 2 { return SingleQubitClifford(0, 2, 1, 4); }
elif xpp == 1 and sp == 3 { return SingleQubitClifford(0, 1, 1, 6); }
